# 网格搜索算法

【参考】
- [cnblogs - 什么是Grid Search 网格搜索？](https://www.cnblogs.com/ysugyl/p/8711205.html)
- [个站 - 网格搜索算法与K折交叉验证](https://ypw.io/GridSearchCV/)

Grid Search：一种调参手段；穷举搜索：在所有候选的参数选择中，通过循环遍历，尝试每一种可能性，表现最好的参数就是最终的结果。其原理就像是在数组里找最大值。（为什么叫网格搜索？以有两个参数的模型为例，参数a有3种可能，参数b有4种可能，把所有可能性列出来，可以表示成一个3×4的表格，其中每个cell就是一个网格，循环过程就像是在每个网格里遍历、搜索，所以叫grid search）：
![image](https://ws4.sinaimg.cn/large/69d4185bly1fwsdj9skrrj20l802ymxw.jpg)

如果我们使用传统的数据集的划分方式，即将数据集划分为训练集和测试集。那么这就存在一个问题，测试集除了用作调整参数，也用来测量模型的好坏；这样做导致最终的评分结果比实际效果要好。（因为测试集在调参过程中，送到了模型里，而我们的目的是将训练模型应用在unseen data上）。
![img](https://ws3.sinaimg.cn/large/69d4185bly1fwsebnwoekj22180r7jsr.jpg)

这就相当于期末考试，平时练习使用了训练试卷，模拟测试使用的是测试试卷，但是期末的时候还是使用的测试试卷，这就没有办法评估出你的成绩到底怎么样，你的成绩好可能就是你之前见过这试卷。

# 验证集

为了避免上面的问题，我们可以把数据集划分为三部分:训练集、验证集、测试集。
![image](https://ws2.sinaimg.cn/large/69d4185bly1fwsey1n3udj20hs0363yq.jpg)

然而，这种grid search方法，其最终的表现好坏与初始数据的划分结果有很大的关系，为了处理这种情况，我们采用交叉验证的方式来减少偶然性。

# K 折验证

在K折交叉验证中，我们用到的数据是**训练集**中的所有数据。我们将训练集的所有数据平均划分成K份（通常选择K=10），取第K份作为验证集，它的作用就像我们用来估计高考分数的模拟题，余下的K-1份作为交叉验证的训练集。

以 SVM 的核函数为例，以 kernel=linear 为例，我们先用第2-10份数据作为训练集训练模型，用第1份数据作为验证集对这次训练的模型进行评分，得到第一个分数；然后重新构建一个 kernel=linear 的SVM，用第1和3-10份数据作为训练集训练模型，用第2份数据作为验证集对这次训练的模型进行评分，得到第二个分数……以此类推，最后构建一个 kernel=linear 的SVM用第1-9份数据作为训练集训练模型，用第10份数据作为验证集对这次训练的模型进行评分，得到第十个分数。于是对于 kernel=linear 的 SVM 模型，我们训练了10次，验证了10次，得到了10个验证分数，然后计算这10个验证分数的平均分数，就是 kernel=linear 的决策树模型的最终验证分数。
![img](https://wx3.sinaimg.cn/large/69d4185bly1fwsf3ujeyyj22gw1skwib.jpg)

对于 kernel=rbf,poly 时，分别进行和 kernel=linear 相同的交叉验证过程，得到它们的最终验证分数。然后我们就可以对这 3 个核函数 SVM 的最终验证分数进行比较，分数最高的那一个就是最优最大深度，对应的模型就是最优模型。

参数的选择是以使用的模型为基础的，如使用 SVM 那么可选的参数就有：kernel、C、gamma等，需要更具框架的具体模型来定。

Grid Search 调参方法存在的共性弊端就是：耗时；参数越多，候选值越多，耗费时间越长！所以，一般情况下，先定一个大范围，然后再细化。整个训练过程如下:
![image](https://ws3.sinaimg.cn/large/69d4185bly1fwsg6dtpuvj20fq07p74z.jpg)

# 示例
以使用 SVM 来处理 鸢尾花 数据为例。

In [7]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import make_scorer, accuracy_score

In [ ]:
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris['data'], iris['target'], train_size=0.8, random_state=0)

parameters = {'kernel':('linear', 'poly', 'rbf'), 'C':[0.1, 1, 10]}
scoring = make_scorer(accuracy_score)
svc = svm.SVC(random_state=0)
# 设置 k 折数
kfold = KFold(n_splits=10)
# 实例化一个 grid search
grid = GridSearchCV(svc, parameters, cv=kfold)
# 找到一个最佳参数，同时使用最佳参数实例化一个新的 SVC
grid = grid.fit(X_train, y_train)

print('best score: %f'%grid.best_score_)
# print('best parameters:')
# coe = grid.best_estimator_
# for key in parameters.keys():
#     print('%s: %s'%(key, coe.get_params()[key]))
    
# 或者使用 besta_params_
print('best parameters:', grid.best_params_)

print('test score: %f'%coe.score(X_test, y_test))

import pandas as pd
pd.DataFrame(grid.cv_results_).T

运行上述代码，可以得到如下结果：
```
best score: 0.958333
best parameters:
kernel: rbf
C: 1
test score: 1.000000
```
![image](https://ws3.sinaimg.cn/large/69d4185bly1fwsfybay6uj20rn0m3446.jpg)